# **Tropos (API Feedback Generation)**
#### *This notebook handles set up to Github/Google Drive, model feedback generation, and generates docx files with feedback.*

## **--Set up**

In [1]:
#######################################################################
# Sets up Google Drive, Github, Github branch
# Installs requirements.txt and needed libraries
#######################################################################
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b fix/RateLimitError origin/fix/RateLimitError

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet #for chatgpt
!pip install -q google-generativeai --quiet #for gemini
!pip install anthropic httpx --quiet #for claude

/content
fatal: destination path 'project' already exists and is not an empty directory.
/content/project
fatal: A branch named 'fix/RateLimitError' already exists.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
ydf 0.11.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.4.1 requires protobuf<4.0.0, but you have protobuf 5.29.4 which is incompatible.
unsloth 2025.4.1 requires protobuf<4.0.0, b

In [2]:
########################################################################
# Default imports and connecting all API keys
#######################################################################
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
import anthropic
openai_key = userdata.get("conner").strip()
deepseek_key = userdata.get("deepseek").strip()
llama_key = userdata.get("groq").strip()
claude_key = userdata.get("claude").strip()


os.environ["OPENAI_API_KEY"] = openai_key

from openai import OpenAI
client = OpenAI(api_key=deepseek_key, base_url="https://api.deepseek.com")

os.environ["LLAMA_API_URL"] = "https://api.groq.com/openai/v1"  # For Ollama
os.environ["LLAMA_API_KEY"] = llama_key  #
os.environ["CLAUDE_API_KEY"] = claude_key

import google.generativeai as genai
genai.configure(api_key=userdata.get("gemini").strip())


## **--Feedback Generation Prep**

In [3]:
#######################################################################
# Ensure past generated output is cleared
# (comment out if do not want)
#######################################################################
from utils.file_utils import clear_directory_if_exists
output_dir = "/content/project/data/generated_output"
clear_directory_if_exists(output_dir)

✅ All files deleted from generated_output.


In [4]:
#######################################################################
# Set up paths for feedback engine
#######################################################################
from tropos import test_feedback_console
reqs_file = "./data/raw/Requirements.docx"
examples_dir = "./data/raw/Student_Submissions"
targets_dir = "./data/unmarked_raw"
outputs_dir = "./data/generated_output"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

## **--Instructor Profile Created Feedback Generator**

In [5]:
# --------------------------------------
# Step 1: Generate Instructor Profile (GPT-4o)
# --------------------------------------

from tropos.models.prompt_builder import generate_full_instructor_profile, load_profile_from_txt

# Paths
examples_dir = "./data/raw/Student_Submissions"
reqs_file = "./data/raw/Requirements.docx"
profile_save_path = "./data/generated_profile/instructor_profile.txt"

batch_size = 7
model_name = "gpt-4o"

# Generate the Instructor Profile
generate_new_profile = True

if generate_new_profile:
    final_profile = generate_full_instructor_profile(
        examples_dir=examples_dir,
        requirements_path=reqs_file,
        batch_size=batch_size,
        model_name=model_name,
        debug=True
    )

    os.makedirs(os.path.dirname(profile_save_path), exist_ok=True)
    with open(profile_save_path, "w", encoding="utf-8") as f:
        f.write(final_profile)

    print(f"📂 Final Instructor Profile saved to {profile_save_path}")
else:
    final_profile = load_profile_from_txt(profile_save_path)
    print("✅ Loaded Instructor Profile successfully!")


📦 Loaded 39 examples
🔹 Processing Batch 1 (7 examples)
📝 Mini Profile 1:
Based on the examples provided, the instructor's feedback habits, tone, priorities, and style can be summarized as follows:

### Feedback Habits:
1. **Focused on Structure and Clarity**: The instructor frequently comments on the need for proper structure, such as using paragraphs effectively and maintaining a clear focus. They emphasize the importance of organizing ideas logically and ensuring that each paragraph serves a distinct purpose.

2. **Encourages Accurate Information**: The instructor is keen on ensuring that students provide current and accurate information. They often point out inaccuracies or misleading statements and encourage students to verify their claims.

3. **Promotes Analysis over Summary**: The instructor consistently urges students to move beyond mere summarization of sources to provide deeper analysis. They highlight the importance of distinguishing between the student’s commentary and the 

In [6]:
from tropos import test_feedback_console
from tropos.models.prompt_builder import load_profile_from_txt

profile_save_path = "./data/generated_profile/instructor_profile.txt"
final_profile = load_profile_from_txt(profile_save_path)

test_feedback_console(
    prompt_type="ProfileShot",
    model="gpt-4o",
    requirements_path="./data/raw/Requirements.docx",
    example_dir="./data/raw/Student_Submissions",
    target_dir="./data/unmarked_raw",
    output_dir="./data/generated_output",
    output_mode="pretty",
    max_examples=None,
    profile_text=final_profile
)

⚠️ Could not inject rubric feedback for Student_1: 'NoneType' object has no attribute 'inject_model_feedback'
--- ProfileShot Feedback for Uncommented_Student_1 ---
📝 Instructor Feedback

--- INLINE FEEDBACK (AT LEAST 4 REQUIRED) ---

"Unfortunately, the year 2019 is known as the year of quarantine. Additionally,
the year 2020 is known for the start of mask mandates." – This opening could be
more engaging. Consider starting with a more specific context or statistic
related to the mask mandate.

"In this paper, it is important to know the difference between a mandate and
recommendation." – This sentence disrupts the flow. Consider integrating this
explanation more smoothly within the paragraph discussing mandates versus
recommendations.

"The viral disease was new with few studies to provide accurate and desperately
needed answers to oftenly sought questions." – "Oftenly" is not standard
English. Consider revising to "often sought" for clarity and correctness.

"According to Nature’s ne

## **--Currently Running Models**

In [ ]:
# # GPT-4o
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4o",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )


## **-- Different Models** (Commented out for now)

In [ ]:
# # GPT-4.1
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4.1",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="raw",
#     max_examples=3
# )

In [ ]:
# # GPT-4.1
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gpt-4.1",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # DeepSeek
# test_feedback_console(
#     prompt_type="FewShot",
#     model="deepseek-chat",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Gemini
# test_feedback_console(
#     prompt_type="FewShot",
#     model="gemini-1.5-pro-latest",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Llama (different prompt_type)
# test_feedback_console(
#     prompt_type="FewShot-Llama",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )

In [ ]:
# # Claude
# test_feedback_console(
#     prompt_type="FewShot",
#     model="claude-3-opus-20240229",
#     requirements_path=reqs,
#     example_dir=examples,
#     target_dir=targets,
#     output_dir=outputs,
#     output_mode="pretty",
#     max_examples=3
# )